# **Data Collection Notebook**

## Objectives

* Fetch monthly data for airport departures and arrivals "Airline On-Time Statistics and Delay Causes" from the United States Department of Transportation (https://www.transtats.bts.gov/)
* Concatenate monthly .csv files into one file
* Preliminary data exploration 

## Inputs

* Monthly csv files for "Airline On-Time Statistics and Delay Causes" - public data

## Outputs

* Generate Dataset: outputs/datasets/collection/airlineDelayPredictor.csv


---

# Set working directory

In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))
current_dir = os.getcwd()
current_dir

'/workspaces/airline-delay-predictor'

# Fetch data from Transtats (USA)

All necessary packages for fetching data have already been install using the requirements.txt file

In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Download zip file '' from https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp?20=E using the following filters:
* Select a carrier: All
* Select an airport All
* Period from: June, 2003
* Period to: June, 2025

File saved in the path : inputs/datasets/raw/transtats_raw_2003june_2025june.zip

Unzip file to get 'Airline_Delay_Cause.csv' and 'Download_Column_Definitions.xlsx'

In [ ]:
DestinationFolder = "inputs/datasets/raw"
filename = "transtats_raw_2003june_2025june.zip"

! unzip {DestinationFolder}/*.zip -d {DestinationFolder}

Archive:  inputs/datasets/raw/transtats_raw_2003june_2025june.zip
replace inputs/datasets/raw/Airline_Delay_Cause.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Fetch data from Kaggle

Configure access token

In [ ]:

import os
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Data downloaded for this url : www.kaggle.com 

Define folders


In [ ]:
# KaggleDatasetPath = "codeinstitute/us-dot-delay-dataset"
KaggleDatasetPath ="usdot/flight-delays"
DestinationFolder = "inputs/datasets/raw"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

Unzip the downloaded file, 

In [ ]:
! unzip {DestinationFolder}/*.zip -d {DestinationFolder} \
  && rm {DestinationFolder}/*.zip \
  && rm kaggle.json

---

# Load and Inspect transtats data

Section 2 content

In [ ]:

import pandas as pd
import numpy as np

df = pd.read_csv(f"inputs/datasets/raw/Airline_Delay_Cause.csv")
df.head()
df.info()

In [ ]:
# flights - too large for pandas
from dask import dataframe as dd

dtype_map = {
    'AIR_TIME': 'float64',
    'ARRIVAL_DELAY': 'float64',
    'ARRIVAL_TIME': 'float64',
    'CANCELLATION_REASON': 'object',
    'DEPARTURE_DELAY': 'float64',
    'DEPARTURE_TIME': 'float64',
    'ELAPSED_TIME': 'float64',
    'SCHEDULED_TIME': 'float64',
    'TAXI_IN': 'float64',
    'TAXI_OUT': 'float64',
    'WHEELS_OFF': 'float64',
    'WHEELS_ON': 'float64'
}

df = dd.read_csv("inputs/datasets/raw/flights.csv", dtype=dtype_map)
df.tail(10)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
642574,2015,12,31,4,B6,1248,N948JB,LAS,JFK,2359,...,1020.0,159.0,0,0,<NA>,0.0,0.0,159.0,0.0,0.0
642575,2015,12,31,4,B6,80,N584JB,RNO,JFK,2359,...,744.0,-21.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642576,2015,12,31,4,B6,802,N589JB,SLC,MCO,2359,...,625.0,17.0,0,0,<NA>,1.0,0.0,16.0,0.0,0.0
642577,2015,12,31,4,B6,98,N607JB,DEN,JFK,2359,...,519.0,-11.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642578,2015,12,31,4,B6,66,N655JB,ABQ,JFK,2359,...,549.0,3.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642579,2015,12,31,4,B6,688,N657JB,LAX,BOS,2359,...,753.0,-26.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642580,2015,12,31,4,B6,745,N828JB,JFK,PSE,2359,...,430.0,-16.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642581,2015,12,31,4,B6,1503,N913JB,JFK,SJU,2359,...,432.0,-8.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642582,2015,12,31,4,B6,333,N527JB,MCO,SJU,2359,...,330.0,-10.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642583,2015,12,31,4,B6,839,N534JB,JFK,BQN,2359,...,442.0,2.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN


/home/cistudent/.local/lib/python3.12/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/home/cistudent/.local/lib/python3.12/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/home/cistudent/.local/lib/python3.12/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


DataFrame Summary

Check columns and nan counts

In [ ]:
df.loc[0:100].compute().info()

<class 'pandas.core.frame.DataFrame'>
Index: 909 entries, 0 to 100
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   YEAR                   909 non-null    int64         
 1   MONTH                  909 non-null    int64         
 2   DAY                    909 non-null    int64         
 3   DAY_OF_WEEK            909 non-null    int64         
 4   AIRLINE                909 non-null    string        
 5   FLIGHT_NUMBER          909 non-null    int64         
 6   TAIL_NUMBER            909 non-null    string        
 7   ORIGIN_AIRPORT         909 non-null    string        
 8   DESTINATION_AIRPORT    909 non-null    string        
 9   SCHEDULED_DEPARTURE    909 non-null    int64         
 10  DEPARTURE_TIME         903 non-null    float64       
 11  DEPARTURE_DELAY        903 non-null    float64       
 12  TAXI_OUT               903 non-null    float64       
 13  WHEELS_OFF

In [ ]:
df.tail(10)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
642574,2015,12,31,4,B6,1248,N948JB,LAS,JFK,2359,...,1020.0,159.0,0,0,<NA>,0.0,0.0,159.0,0.0,0.0
642575,2015,12,31,4,B6,80,N584JB,RNO,JFK,2359,...,744.0,-21.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642576,2015,12,31,4,B6,802,N589JB,SLC,MCO,2359,...,625.0,17.0,0,0,<NA>,1.0,0.0,16.0,0.0,0.0
642577,2015,12,31,4,B6,98,N607JB,DEN,JFK,2359,...,519.0,-11.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642578,2015,12,31,4,B6,66,N655JB,ABQ,JFK,2359,...,549.0,3.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642579,2015,12,31,4,B6,688,N657JB,LAX,BOS,2359,...,753.0,-26.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642580,2015,12,31,4,B6,745,N828JB,JFK,PSE,2359,...,430.0,-16.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642581,2015,12,31,4,B6,1503,N913JB,JFK,SJU,2359,...,432.0,-8.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642582,2015,12,31,4,B6,333,N527JB,MCO,SJU,2359,...,330.0,-10.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN
642583,2015,12,31,4,B6,839,N534JB,JFK,BQN,2359,...,442.0,2.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN


### Derived additional variables

In [3]:
df["date"] = dd.to_datetime(
    df["YEAR"].astype(str) + "-" +
    df["MONTH"].astype(str) + "-" +
    df["DAY"].astype(str),
    errors="coerce"
)

# Boolean delay indicators
df["flight_depart_delayed"] = (df["DEPARTURE_DELAY"] > 15).astype(int)
df["flight_arrive_delayed"] = (df["ARRIVAL_DELAY"] > 15).astype(int)

In [4]:
# Columns 1- 10
df.iloc[:, :10].head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10
2,2015,1,1,4,US,840,N171US,SFO,CLT,20
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25


In [ ]:
# Columns 11- 20
df.iloc[:, 11:21].head()

,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL
0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430
1,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750
2,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806
3,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805
4,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320


In [ ]:
# Columns 21- 30
df.iloc[:, 21:].head()

,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,date,flight_depart_delayed,flight_arrive_delayed
0,408.0,-22.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,0
1,741.0,-9.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,0
2,811.0,5.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,0
3,756.0,-9.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,0
4,259.0,-21.0,0,0,<NA>,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,0


Round floats to 2 decimal places

In [ ]:
df = df.round(2)
df.head(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,date,flight_depart_delayed,flight_arrive_delayed
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,0,<NA>,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,0,<NA>,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,0
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,0,<NA>,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,0,<NA>,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,0
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,0,<NA>,NaN,NaN,NaN,NaN,NaN,2015-01-01,0,0


/home/cistudent/.local/lib/python3.12/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/home/cistudent/.local/lib/python3.12/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/home/cistudent/.local/lib/python3.12/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/home/cistudent/.local/lib/python3.12/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


### Sample df evenly across months, airlines, days of week and scheduled time

In [ ]:
# df_sample = (
#     df.groupby(["MONTH", "AIRLINE", "DAY_OF_WEEK","ORIGIN_AIRPORT"], group_keys=False)
#            .apply(lambda x: x.sample(frac=40000 / len(df), random_state=42))
# )

random_state= 131

df_sample = df.sample(frac=10_000 / len(df), random_state=42)


# Save file 

create destination folder: outputs/datasets/collection

path: outputs/dataset/collection as transtatsAirlineDelay.csv

In [9]:
import os

output_path = r"outputs/datasets/collection"
try:
  os.makedirs(name=output_path)
except Exception as e:
  print(e)

df_sample.to_csv(f"outputs/datasets/collection/usDOTflightDelay_*.csv",index=False)

[Errno 17] File exists: 'outputs/datasets/collection'


['/workspaces/airline-delay-predictor/outputs/datasets/collection/usDOTflightDelay_0.csv',
 '/workspaces/airline-delay-predictor/outputs/datasets/collection/usDOTflightDelay_1.csv',
 '/workspaces/airline-delay-predictor/outputs/datasets/collection/usDOTflightDelay_2.csv',
 '/workspaces/airline-delay-predictor/outputs/datasets/collection/usDOTflightDelay_3.csv',
 '/workspaces/airline-delay-predictor/outputs/datasets/collection/usDOTflightDelay_4.csv',
 '/workspaces/airline-delay-predictor/outputs/datasets/collection/usDOTflightDelay_5.csv',
 '/workspaces/airline-delay-predictor/outputs/datasets/collection/usDOTflightDelay_6.csv',
 '/workspaces/airline-delay-predictor/outputs/datasets/collection/usDOTflightDelay_7.csv',
 '/workspaces/airline-delay-predictor/outputs/datasets/collection/usDOTflightDelay_8.csv']

---

# Conclusions and Next Steps

* raw data have been downloaded
* Preliminary checks and variable data type changes
* Output file saved to the 'collections' subfolder in 'outputs/dataset/'
* Next notebook will tackle data exploration